In [ ]:
### Compute roughness based on a digital terrain model ###

library(raster)
library(rgdal)
library(SDMTools)

# preprocess JAXA ALOS
# Use the "AVERAGE" data
dtm_files <- list.files('', pattern = 'DSM.tif$', full.names = T)
dtm_list <- c()
for(file in dtm_files) {
  temp <- raster(file)
  dtm_list <- append(dtm_list, temp)
}
rm(dtm_files, file, temp)

# Studyarea
sa <- readOGR('')

dtm_list.mosaicargs <- dtm_list
dtm_list.mosaicargs$fun <- mean
dtm_mos <- do.call(mosaic, dtm_list.mosaicargs)

dtm_mos <- projectRaster(dtm_mos, crs = sa@proj4string)
dtm <- crop(dtm_mos, sa)
rm(dtm_list, dtm_list.mosaicargs, dtm_mos)

# writeRaster(dtm, '', format = 'GTiff', overwrite = T)

m <- matrix(1, nrow = 3, ncol = 3)

# method 3 from http://gis4geomorphology.com/roughness-topographic-position/
dtm_mean <- focal(dtm, m, fun = mean)
dtm_min <- focal(dtm, m, fun = min)
dtm_max <- focal(dtm, m, fun = max)
dtm_range <- dtm_max - dtm_min

dtm_sd <- overlay(dtm, dtm_mean, dtm_range, m, fun = function(original, mean, range) {(mean - original)/range})
#writeRaster(dtm_sd, '', format = 'GTiff', overwrite = T)
rm(dtm_max, dtm_min, dtm_mean, dtm_range)
dtm_sd2 <- focal(dtm_sd, m, fun = sd)
writeRaster(dtm_sd2, '', format = 'GTiff', overwrite = T)

In [ ]:
### Average roughness per cell ###

library(raster)
library(rgdal)

grid <- readOGR('')

# Roughness index
rough <- raster('')

grid_r <- extract(rough, grid, method = 'bilinear', fun = mean, na.rm = T, sp = T)
names(grid500_r) <- c('FID', 'roughind')
writeOGR(grid500_r, dsn = '', layer = '', driver = 'ESRI Shapefile', overwrite_layer = T)